In [1]:
import torch
import time
from snn_delays.experimental_models.snn_delay_prun import P_DelaySNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils import train, get_device
from snn_delays.utils.test_behavior import tb_save_max_last_acc

'''
SHD dataset as in ablation study
'''

device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'shd'
total_time = 50
batch_size = 1024

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time,
                  crop_to=1e6)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()
          
num_epochs = 50

lr = 1e-3
# SNN CON DELAYS
taimu1 = time.time()

tau_m = 'normal'
ckpt_dir = 'exp3_shd50_rnn' 

snn = P_DelaySNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=(40,1), delay_type='h', tau_m = tau_m, delay_mask='random', n_pruned_delays=3,
    win=total_time, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.to(device)

c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda:0
[CropTime(min=0, max=1000000.0), ToFrame(sensor_size=(700, 1, 1), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]
<class 'list'>

[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39])

[INFO] Delays o: tensor([0])
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918


P_DelaySNN(
  (criterion): CrossEntropyLoss()
  (f0_f1): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): FixedMaskedLinear(
    (linear): Linear(in_features=2560, out_features=64, bias=False)
  )
  (f2_o): Linear(in_features=64, out_features=20, bias=False)
)

In [2]:
snn.f1_f2.mask

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False]], device='cuda:0')

In [3]:
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

training shd50_l2_40d1.t7 for 50 epochs...
Epoch [1/50], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Step [2/7], Loss: 2.98397
l1_score: 0
Step [4/7], Loss: 2.94775
l1_score: 0
Step [6/7], Loss: 2.87554
l1_score: 0
Step [8/7], Loss: 2.77475
l1_score: 0
Time elasped: 43.63341546058655
Test Loss: 2.6449894110361734
Avg spk_count per neuron for all 50 time-steps 4.0717997550964355
Avg spk per neuron per layer [13.036364012809187, 3.250835081713781]
Test Accuracy of the model on the test samples: 16.652

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp3_shd50_rnn
saving max acc: 16.651943462897528
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp3_shd50_rnn
Epoch [2/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 2.60721
l1_score: 0
Step [4/7], Loss: 2.43981
l1_score: 0
Step [6/7], Loss: 2.36374
l1_score: 0
Step [8/7], Loss: 2.19655
l1_score: 0
Time elasped: 3.2845234870910645
Test Loss: 2.0090959469477334
Avg spk_count per neuron for all 50 time-steps 4.903306484222412
Avg spk per neuron per

In [4]:
snn.f1_f2.mask

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False]], device='cuda:0')

In [8]:
snn.f1_f2.linear.weight

Parameter containing:
tensor([[-0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [-0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0000,  0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000, -0.0000,  0.0000,  ..., -0.0073,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0256,  ..., -0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0016, -0.0000]],
       device='cuda:0', requires_grad=True)